In [11]:
import tensorflow as tf
import helper
from data_generator import CoupletsDataGenerator
import numpy as np

ModuleNotFoundError: No module named 'tensorflow.nn'

In [2]:
%cd ..

/Users/zytec/Documents/.zhangshulin/dl/couplets


In [3]:
vocab_size, index2word, word2index, train_set, dev_set, test_set = helper.process_dataset()

vocab_size: 729587
train_set shape: (1540982, 52)
dev_set shape: (4000, 40)
test_set shape: (4000, 43)


In [4]:
train_gen = CoupletsDataGenerator(train_set)
dev_gen = CoupletsDataGenerator(dev_set)
test_gen = CoupletsDataGenerator(test_set)

In [5]:
def build_inputs():
    with tf.name_scope('input_placeholders'):
        input = tf.placeholder(shape=(None, None), dtype=tf.int32, name='input')
        label = tf.placeholder(shape=(None, None), dtype=tf.int32, name='label')
        keep_prob = tf.placeholder(dtype=tf.float32, name='keep_prob')
        
    return input, label, keep_prob

In [6]:
def embeding_layer(input, vocab_size, embeding_dim):
    with tf.name_scope('embeding_layer'):
        embeding_matrix = tf.Variable(tf.random_uniform(shape=(vocab_size, embeding_dim),
                                                        minval=-1, maxval=1, dtype=tf.float32),
                                     name='embeding_matrix')
        embed = tf.nn.embedding_lookup(embeding_matrix, input, name='embed_lookup')
        
    return embed

In [13]:
def build_lstm_cell(num_units, num_layers, keep_prob):
    with tf.name_scope('lstm_cell'):
        def lstm():
            lstm = tf.nn.rnn_cell.BasicLSTMCell(num_units)
            dropout = tf.nn.rnn_cell.DropoutWrapper(lstm, output_keep_prob=keep_prob)
            return dropout
        
        cell = tf.nn.rnn_cell.MultiRNNCell([lstm() for _ in range(num_layers)])
        
    return cell

In [10]:
def lstm_layer(embed_input, num_units, num_layers, keep_prob):
    with tf.name_scope('lstm_layer'):
        

SyntaxError: unexpected EOF while parsing (<ipython-input-10-3235f8258cc1>, line 3)